## Data Processing

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

Listing all "NA" columns in training dataset...

In [3]:
list_na = [k[0] for k in df.isna().any().to_dict().items() if k[1] == True]
print(list_na)

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


## Fill Missing Values

There are certain cells whose "NA" values will take "No" values per the dataset description.

Listing all such columns...

In [4]:
list_no = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
          'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
          'Fence']

In [5]:
# Checking if list_no defined above is a subset of list_na 
set(list_no).issubset(set(list_na))

True

In [6]:
df_tr = df.copy()

for j in list_no:
    df_tr[j].fillna("No", inplace=True)
    df_test[j].fillna("No", inplace=True)

From above processing, few columns in "NA" list have been filled with "No" values per list list_no. Now, we will address missing values in remaining columns.

Getting list of remaining columns...

In [7]:
list_na_ = list(set(list_na) - set(list_no))
list_na_

['GarageYrBlt',
 'MasVnrType',
 'MasVnrArea',
 'LotFrontage',
 'Electrical',
 'MiscFeature']

In [8]:
# Printing datatype of remaining columns with "NA" values
for k in list_na_:
    print(df_tr[k].dtype)

float64
object
float64
float64
object
object


If column is of type "float64" or "int64", we will replace the null values by the mean of the column. 

For other datatypes, we will fill by mode.

Filling Null values in Training dataset...

In [9]:
for k in list_na_:
    if (df_tr[k].dtype =='float64') or (df_tr[k].dtype =='int64'):
        df_tr[k].fillna(df_tr[k].mean(), inplace=True)
    else:
        df_tr[k].fillna(df_tr[k].mode()[0], inplace=True)

We had filled "No" values in test dataset.

Filling null values for remaining columns in test dataset...

In [10]:
for k in df_test.columns:
    if df_test[k].dtype =='float64' or df_test[k].dtype =='int64':
        df_test[k].fillna(df_test[k].mean(), inplace=True)
    else:
        df_test[k].fillna(df_test[k].mode()[0], inplace=True)

Checking if there is remaining null values in integer type columns in training dataset.

In [11]:
list_int = [k for k in df_tr.columns if df_tr[k].dtype=='int64']
df_tr[list_int].isna().any().sum()

0

In [12]:
df_tr.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,22.024023,9981.264932,1.382997,1.112799,30.202904,20.645407,180.569112,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,60.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,70.049958,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [13]:
list_int_float = [j for j in df_tr.columns if (df_tr[j].dtype=='float64') or 
                  (df_tr[j].dtype=='int64')]

df_tr[list_int_float].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   OverallQual    1460 non-null   int64  
 5   OverallCond    1460 non-null   int64  
 6   YearBuilt      1460 non-null   int64  
 7   YearRemodAdd   1460 non-null   int64  
 8   MasVnrArea     1460 non-null   float64
 9   BsmtFinSF1     1460 non-null   int64  
 10  BsmtFinSF2     1460 non-null   int64  
 11  BsmtUnfSF      1460 non-null   int64  
 12  TotalBsmtSF    1460 non-null   int64  
 13  1stFlrSF       1460 non-null   int64  
 14  2ndFlrSF       1460 non-null   int64  
 15  LowQualFinSF   1460 non-null   int64  
 16  GrLivArea      1460 non-null   int64  
 17  BsmtFullBath   1460 non-null   int64  
 18  BsmtHalf

Finding correlation of integer and float type columns in training dataset.

In [14]:
dict_corr = df_tr[list_int_float].corr()['SalePrice'].to_dict()
print(dict_corr)

{'Id': -0.021916719443430967, 'MSSubClass': -0.08428413512659509, 'LotFrontage': 0.3349008515405512, 'LotArea': 0.2638433538714051, 'OverallQual': 0.7909816005838053, 'OverallCond': -0.07785589404867797, 'YearBuilt': 0.5228973328794967, 'YearRemodAdd': 0.5071009671113866, 'MasVnrArea': 0.475241316773666, 'BsmtFinSF1': 0.3864198062421535, 'BsmtFinSF2': -0.011378121450215146, 'BsmtUnfSF': 0.21447910554696928, 'TotalBsmtSF': 0.6135805515591943, '1stFlrSF': 0.6058521846919153, '2ndFlrSF': 0.31933380283206736, 'LowQualFinSF': -0.02560613000067955, 'GrLivArea': 0.7086244776126515, 'BsmtFullBath': 0.22712223313149427, 'BsmtHalfBath': -0.01684415429735895, 'FullBath': 0.5606637627484453, 'HalfBath': 0.28410767559478256, 'BedroomAbvGr': 0.16821315430073963, 'KitchenAbvGr': -0.13590737084214105, 'TotRmsAbvGrd': 0.5337231555820284, 'Fireplaces': 0.46692883675152763, 'GarageYrBlt': 0.47017745374048125, 'GarageCars': 0.6404091972583519, 'GarageArea': 0.6234314389183622, 'WoodDeckSF': 0.324413444568

In [15]:
dict_corr_ = {}
for k in dict_corr.items():
    if np.abs(k[1])>=0.5:
        dict_corr_[k[0]] = k[1]

print(dict_corr_)
print("-------------------------------------------------\n")
list_corr = [k[0] for k in dict_corr_.items()]
print("Printing below all Keys from above dictionary:\n")
print(list_corr)

{'OverallQual': 0.7909816005838053, 'YearBuilt': 0.5228973328794967, 'YearRemodAdd': 0.5071009671113866, 'TotalBsmtSF': 0.6135805515591943, '1stFlrSF': 0.6058521846919153, 'GrLivArea': 0.7086244776126515, 'FullBath': 0.5606637627484453, 'TotRmsAbvGrd': 0.5337231555820284, 'GarageCars': 0.6404091972583519, 'GarageArea': 0.6234314389183622, 'SalePrice': 1.0}
-------------------------------------------------

Printing below all Keys from above dictionary:

['OverallQual', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea', 'SalePrice']


We had filtered out columns based on correlation with output variable "SalePrice". These columns were integer or float datatypes. We will now evaluate what all columns to keep removing columns with low correlation with output variable.

In [16]:
list_drop = list(set(list_int_float) - set(list_corr))
list_col_keep = list(set(list(df_tr.columns)) - set(list_drop))
print(list_col_keep)

['Condition1', 'GarageQual', 'RoofStyle', 'BsmtQual', 'ExterCond', 'GrLivArea', 'BldgType', 'BsmtExposure', 'HouseStyle', 'PavedDrive', 'BsmtFinType2', 'GarageFinish', 'CentralAir', 'SaleType', 'MasVnrType', 'Exterior1st', 'OverallQual', 'SalePrice', 'GarageCars', 'Heating', 'LandContour', 'Alley', 'FullBath', 'BsmtCond', 'SaleCondition', 'Utilities', 'Foundation', 'LotConfig', 'FireplaceQu', 'GarageCond', 'HeatingQC', 'ExterQual', 'RoofMatl', 'Functional', '1stFlrSF', 'Exterior2nd', 'TotRmsAbvGrd', 'PoolQC', 'GarageType', 'Street', 'LotShape', 'YearRemodAdd', 'YearBuilt', 'KitchenQual', 'MSZoning', 'Fence', 'TotalBsmtSF', 'Electrical', 'Neighborhood', 'GarageArea', 'BsmtFinType1', 'MiscFeature', 'Condition2', 'LandSlope']


In [17]:
df_tr_ = df_tr[list_col_keep]

Ordering test dataset similar to training dataset...

In [18]:
list_ = [j for j in list_col_keep if j!='SalePrice']
list_.insert(0, 'Id')
df_test_ = df_test[list_]

Making list of category columns in training dataset...

In [19]:
cat_col = [k for k in list_col_keep if df_tr_[k].dtype=="object"]
print(cat_col)

['Condition1', 'GarageQual', 'RoofStyle', 'BsmtQual', 'ExterCond', 'BldgType', 'BsmtExposure', 'HouseStyle', 'PavedDrive', 'BsmtFinType2', 'GarageFinish', 'CentralAir', 'SaleType', 'MasVnrType', 'Exterior1st', 'Heating', 'LandContour', 'Alley', 'BsmtCond', 'SaleCondition', 'Utilities', 'Foundation', 'LotConfig', 'FireplaceQu', 'GarageCond', 'HeatingQC', 'ExterQual', 'RoofMatl', 'Functional', 'Exterior2nd', 'PoolQC', 'GarageType', 'Street', 'LotShape', 'KitchenQual', 'MSZoning', 'Fence', 'Electrical', 'Neighborhood', 'BsmtFinType1', 'MiscFeature', 'Condition2', 'LandSlope']


In [20]:
cat_col_ordinal = ['LandContour', 'Condition2', 'HeatingQC', 'MSZoning', 'Neighborhood', 
                   'RoofMatl', 'Utilities', 'MasVnrType', 'KitchenQual', 'Exterior2nd', 
                    'BsmtCond', 'Exterior1st', 'HouseStyle', 
                   'Electrical', 'Fence', 'GarageQual', 'Alley', 'FireplaceQu', 'Foundation', 
                   'MiscFeature', 'RoofStyle', 'GarageFinish','LotShape', 
                   'GarageType', 'SaleCondition', 'BsmtFinType1', 'PavedDrive', 'Heating', 
                   'LandSlope', 'ExterQual', 'ExterCond', 'PoolQC', 'Condition1', 
                   'BsmtExposure', 'GarageCond', 'LotConfig', 'Functional', 
                   'BsmtFinType2', 'BsmtQual']

cat_col_label = ['Street' ,'BldgType', 'SaleType', 'CentralAir']

In [21]:
from sklearn.preprocessing import OrdinalEncoder


for j in cat_col_ordinal:
    print("Ordinal Encoding starting for column",j)
    OE = OrdinalEncoder(categories=[list(set(list(df_tr_.loc[:,j].unique())).
                                         union(set(list(df_test_.loc[:,j].unique()))))])
    df_tr_.loc[:,j] = OE.fit_transform(df_tr_[[j]])
    df_test_.loc[:,j] = OE.transform(df_test_[[j]])
    print("Ordinal Encoding done for column",j)
    print("---------------------------------------")


Ordinal Encoding starting for column LandContour
Ordinal Encoding done for column LandContour
---------------------------------------
Ordinal Encoding starting for column Condition2
Ordinal Encoding done for column Condition2
---------------------------------------
Ordinal Encoding starting for column HeatingQC
Ordinal Encoding done for column HeatingQC
---------------------------------------
Ordinal Encoding starting for column MSZoning
Ordinal Encoding done for column MSZoning
---------------------------------------
Ordinal Encoding starting for column Neighborhood
Ordinal Encoding done for column Neighborhood
---------------------------------------
Ordinal Encoding starting for column RoofMatl
Ordinal Encoding done for column RoofMatl
---------------------------------------
Ordinal Encoding starting for column Utilities
Ordinal Encoding done for column Utilities
---------------------------------------
Ordinal Encoding starting for column MasVnrType
Ordinal Encoding done for column M

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

Ordinal Encoding done for column BsmtExposure
---------------------------------------
Ordinal Encoding starting for column GarageCond
Ordinal Encoding done for column GarageCond
---------------------------------------
Ordinal Encoding starting for column LotConfig
Ordinal Encoding done for column LotConfig
---------------------------------------
Ordinal Encoding starting for column Functional
Ordinal Encoding done for column Functional
---------------------------------------
Ordinal Encoding starting for column BsmtFinType2
Ordinal Encoding done for column BsmtFinType2
---------------------------------------
Ordinal Encoding starting for column BsmtQual
Ordinal Encoding done for column BsmtQual
---------------------------------------


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [22]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()


for col in cat_col_label:
    print("Label encoding for",col, "column")
    df_tr_.loc[:,col] = encoder.fit_transform(df_tr_.loc[:,col])
    df_test_.loc[:,col]  = encoder.fit_transform(df_test_.loc[:,col])
    print("Encoding done for",col,"column")
    print("------------------------------")
        

Label encoding for Street column
Encoding done for Street column
------------------------------
Label encoding for BldgType column
Encoding done for BldgType column
------------------------------
Label encoding for SaleType column
Encoding done for SaleType column
------------------------------
Label encoding for CentralAir column
Encoding done for CentralAir column
------------------------------


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Check if there is any missing values in Training and Test dataset.

In [23]:
df_tr_.isna().any().sum(), df_test_.isna().any().sum()

(0, 0)

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [25]:
y = df_tr_['SalePrice']
X = df_tr_.drop('SalePrice', axis=1)

In [26]:
model = RandomForestRegressor(max_leaf_nodes=100)
model.fit(X, y)
score = cross_val_score(model, X, y, cv=10)
print(score.mean())

0.8481849258136103


Extracting test id from test dataset...

In [27]:
test_id = df_test_['Id']
print(test_id)

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64


Reordering test dataset columns per training dataset feeded to RandomForest model.

In [28]:
df_test_ = df_test_[X.columns]

In [29]:
saleprice = model.predict(df_test_)
print(saleprice)

[123281.13110154 150433.526782   172570.70643702 ... 143361.50463565
 117619.46986105 223183.6404189 ]


In [30]:
submission = pd.DataFrame({
    "Id": test_id,
    "SalePrice": saleprice
})

submission.to_csv("submission_shivam9400.csv", index=False)
submission.sample(10)

,Id,SalePrice
863,2324,167020.190580
792,2253,157310.119644
1154,2615,152685.487641
731,2192,88635.137622
235,1696,251573.890219
383,1844,135398.158303
267,1728,184571.078957
1071,2532,229170.536624
686,2147,156313.132868
763,2224,213058.945408
